# CBOW model for text classification
We develop a continuous bag of words (CBOW) model for a text classification task described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

In [3]:
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [4]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [5]:
unpack_dataset()

--2022-02-28 12:37:37--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519599 (507K) [application/x-gzip]
Saving to: ‘rotten_imdb.tar.gz.2’

rotten_imdb.tar.gz. 100%[===================>] 507.42K  1.08MB/s    in 0.5s    

2022-02-28 12:37:38 (1.08 MB/s) - ‘rotten_imdb.tar.gz.2’ saved [519599/519599]

mkdir: data: File exists
x quote.tok.gt9.5000
x plot.tok.gt9.5000
x subjdata.README.1.0


In [6]:
!ls data

1000.mmf            21000.mmf           32965.mmf           glove.6B.zip
10000.mmf           22000.mmf           4000.mmf            hour.csv
11000.mmf           23000.mmf           5000.mmf            lg32965.zip
12000.mmf           24000.mmf           6000.mmf            names_test.csv
13000.mmf           25000.mmf           7000.mmf            names_train.csv
14000.mmf           26000.mmf           8000.mmf            plot.tok.gt9.5000
15000.mmf           27000.mmf           9000.mmf            quote.tok.gt9.5000
16000.mmf           28000.mmf           Readme.txt          subjdata.README.1.0
17000.mmf           29000.mmf           day.csv             train.csv
18000.mmf           3000.mmf            glove.6B.100d.txt   train.csv.zip
19000.mmf           30000.mmf           glove.6B.200d.txt
2000.mmf            31000.mmf           glove.6B.300d.txt
20000.mmf           32000.mmf           glove.6B.50d.txt


In [7]:
! head -2 data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 


In [8]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/glove.6B.300d.txt'),
 PosixPath('data/1000.mmf'),
 PosixPath('data/18000.mmf'),
 PosixPath('data/14000.mmf'),
 PosixPath('data/22000.mmf'),
 PosixPath('data/30000.mmf'),
 PosixPath('data/16000.mmf'),
 PosixPath('data/32000.mmf'),
 PosixPath('data/20000.mmf'),
 PosixPath('data/glove.6B.100d.txt'),
 PosixPath('data/names_train.csv'),
 PosixPath('data/3000.mmf'),
 PosixPath('data/24000.mmf'),
 PosixPath('data/28000.mmf'),
 PosixPath('data/12000.mmf'),
 PosixPath('data/7000.mmf'),
 PosixPath('data/names_test.csv'),
 PosixPath('data/9000.mmf'),
 PosixPath('data/glove.6B.50d.txt'),
 PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/5000.mmf'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/26000.mmf'),
 PosixPath('data/10000.mmf'),
 PosixPath('data/23000.mmf'),
 PosixPath('data/31000.mmf'),
 PosixPath('data/19000.mmf'),
 PosixPath('data/quote.tok.gt9.5000'),
 PosixPath('data/15000.mmf'),
 PosixPath('data/hour.csv'),
 PosixPath('data/glove.6B.200d.txt'),
 PosixP

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [9]:
# We need each line in the file 
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [10]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [11]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [12]:
np.array(obj_lines[0].strip().lower().split(" "))

array(['the', 'movie', 'begins', 'in', 'the', 'past', 'where', 'a',
       'young', 'boy', 'named', 'sam', 'attempts', 'to', 'save', 'celebi',
       'from', 'a', 'hunter', '.'], dtype='<U8')

### Better tokenization with Spacy

In [13]:
#!pip install -U spacy

In [24]:
import spacy

In [25]:
# first time run this
#!python3 -m spacy download en_core_web_sm

In [26]:
tok = spacy.load('en_core_web_sm')

In [27]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [28]:
len(obj_lines)

5000

In [29]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [30]:
test = tok(obj_lines[0])

In [31]:
np.array([x for x in test])

array([the, movie, begins, in, the, past, where, a, young, boy, named,
       sam, attempts, to, save, celebi, from, a, hunter, ., 
], dtype=object)

## Split dataset in train and validation

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [34]:
X[0], y[0]

('smart and alert , thirteen conversations about one thing is a small gem .',
 0.0)

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'),
 array([1., 0., 0., 1., 1.]))

## Word to index mapping
In interest of time we will tokenize without spaCy. Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [37]:
from collections import defaultdict

In [38]:
def get_vocab(content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab      

In [39]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [40]:
#word_count

In [41]:
len(word_count.keys())

21415

In [42]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [43]:
len(word_count.keys())

4065

In [44]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [45]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [46]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_val_len = np.array([len(x.split()) for x in X_val])

In [47]:
np.percentile(x_train_len, 95) # let set the max sequence len to N=40

43.0

In [48]:
X_train[0]

'will god let her fall or give her a new path ?'

In [49]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

7

In [50]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([ 6,  4,  9, 11, 10,  3,  2, 11,  5, 12,  8,  7])

In [51]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [52]:
encode_sentence(X_train[0])

array([ 6,  4,  9, 11, 10,  3,  2, 11,  5, 12,  8,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [53]:
x_train_len = np.minimum(x_train_len, 40)
x_val_len = np.minimum(x_val_len, 40)

In [54]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [55]:
x_val = np.vstack([encode_sentence(x) for x in X_val])
x_val.shape

(2000, 40)

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors. 

In [56]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.4693, -0.0340, -0.4917, -0.2921],
        [-1.1156,  1.8495, -0.5810, -0.8816],
        [-0.2286,  0.4492, -0.0050, -0.7076],
        [-0.9398,  2.1690, -0.6302,  1.5879],
        [-1.4928, -1.0337,  0.5152,  0.5287],
        [ 2.0545,  1.1455,  0.1970,  0.4264],
        [ 1.8367,  0.0758, -0.2227,  0.9991],
        [-0.3068,  1.2297,  0.2581, -0.1379],
        [-0.5399, -1.5186, -0.5054, -0.1748]], requires_grad=True)

Note that the `padding_idx` has embedding vector 0.

In [57]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

tensor([[[-0.4693, -0.0340, -0.4917, -0.2921],
         [-0.9398,  2.1690, -0.6302,  1.5879],
         [-0.4693, -0.0340, -0.4917, -0.2921],
         [-1.4928, -1.0337,  0.5152,  0.5287],
         [-0.4693, -0.0340, -0.4917, -0.2921],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence. 

In [58]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [59]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [60]:
embed(a).shape

torch.Size([2, 3, 4])

In [61]:
embed(a)

tensor([[[-0.4693, -0.0340, -0.4917, -0.2921],
         [-0.9398,  2.1690, -0.6302,  1.5879],
         [-0.4693, -0.0340, -0.4917, -0.2921]],

        [[-0.4693, -0.0340, -0.4917, -0.2921],
         [-0.2286,  0.4492, -0.0050, -0.7076],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

In [62]:
embed(a).sum(dim=1)

tensor([[-1.8783,  2.1011, -1.6137,  1.0037],
        [-0.6979,  0.4152, -0.4967, -0.9997]], grad_fn=<SumBackward1>)

In [63]:
sum_embs = embed(a).sum(dim=1) 
sum_embs/ s.view(s.shape[0], 1)

tensor([[-0.6261,  0.7004, -0.5379,  0.3346],
        [-0.3489,  0.2076, -0.2484, -0.4998]], grad_fn=<DivBackward0>)

## Continuous Bag of Words Model

In [64]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear = nn.Linear(emb_size, 1)
        
    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s
        x = self.linear(x)
        return x

In [65]:
model = CBOW(vocab_size=5, emb_size=3)

In [66]:
model.word_emb.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.2730,  0.0305,  0.8006],
        [-0.9890,  1.6663, -0.0798],
        [-0.3093, -1.7463, -0.3532],
        [ 0.0774,  0.6674, -1.6750]], requires_grad=True)

In [67]:
s = s.view(s.shape[0], 1)
model(a, s)

tensor([[ 0.1662],
        [-0.4649]], grad_fn=<AddmmBackward>)

# Training the CBOW model 

In [68]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

4067


In [69]:
def val_metrics(model):
    model.eval()
    x = torch.LongTensor(x_val) #.cuda()
    y = torch.Tensor(y_val).unsqueeze(1) #).cuda()
    s = torch.Tensor(x_val_len).view(x_val_len.shape[0], 1)
    y_hat = model(x, s)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [70]:
# accuracy of a random model should be around 0.5
val_metrics(model)

(0.7035585641860962, 0.49050000309944153)

In [72]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        model.train()
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        s = torch.Tensor(x_train_len).view(x_train_len.shape[0], 1)
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss, val_accuracy = val_metrics(model)
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % 
              (loss.item(), val_loss, val_accuracy))

In [73]:
model = CBOW(vocab_size=V, emb_size=50)
train_epocs(model, epochs=10, lr=0.1)

train_loss 0.694 val_loss 0.651 val_accuracy 0.577
train_loss 0.644 val_loss 0.556 val_accuracy 0.801
train_loss 0.546 val_loss 0.464 val_accuracy 0.840
train_loss 0.442 val_loss 0.371 val_accuracy 0.868
train_loss 0.336 val_loss 0.314 val_accuracy 0.872
train_loss 0.264 val_loss 0.273 val_accuracy 0.888
train_loss 0.209 val_loss 0.259 val_accuracy 0.893
train_loss 0.178 val_loss 0.256 val_accuracy 0.900
train_loss 0.153 val_loss 0.260 val_accuracy 0.899
train_loss 0.132 val_loss 0.272 val_accuracy 0.904


In [74]:
train_epocs(model, epochs=10, lr=0.01)

train_loss 0.116 val_loss 0.268 val_accuracy 0.905
train_loss 0.111 val_loss 0.267 val_accuracy 0.906
train_loss 0.107 val_loss 0.267 val_accuracy 0.906
train_loss 0.104 val_loss 0.267 val_accuracy 0.906
train_loss 0.101 val_loss 0.267 val_accuracy 0.907
train_loss 0.098 val_loss 0.267 val_accuracy 0.907
train_loss 0.095 val_loss 0.267 val_accuracy 0.907
train_loss 0.092 val_loss 0.268 val_accuracy 0.908
train_loss 0.090 val_loss 0.269 val_accuracy 0.908
train_loss 0.087 val_loss 0.270 val_accuracy 0.906


# Data loaders for SGD

Nearly all of deep learning is powered by one very important algorithm: **stochastic gradient descent (SGD)**. SGD can be seeing as an approximation of **gradient descent** (GD). In GD you have to run through *all* the samples in your training set to do a single itaration. In SGD you use *only one* or *a subset*  of training samples to do the update for a parameter in a particular iteration. The subset use in every iteration is called a **batch** or **minibatch**.

In [66]:
from torch.utils.data import Dataset, DataLoader

Next we are going to create a data loader. The data loader provides the following features:
* Batching the data
* Shuffling the data
* Load the data in parallel using multiprocessing workers.

In [67]:
def encode_sentence2(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [68]:
encode_sentence2(X_train[0])

(array([11,  3,  5,  7,  4, 12,  6,  7,  8, 10,  2,  9,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0], dtype=int32),
 12)

In [126]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence2(x)
        return x, self.y[idx], s
    
sub_dataset_train = SubjectivityDataset(X_train, y_train)

In [127]:
train_loader = DataLoader(sub_dataset_train, batch_size=5, shuffle=True)
x, y, s = next(iter(train_loader))

In [128]:
x, y, s

(tensor([[  19,   19,   19, 3628, 1207,   14,  681,    1,   52,  159, 1167,   18,
           594,    1,   37, 3218,    8,  113,    1,  131,   52,  117,   18, 2633,
            19,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [ 952, 3801,    1,   26,   14,  149,   72,  415,    8,    1,   66,   14,
          2087,    1,  109,   98,  211,    1,  175,   37,  931,   61,   38,   88,
            14,  373,   19,   30,  245,   33,   26,   14,  239,    1,    1, 3837,
            88,    1, 1077,  973],
         [  81,    1,   26,    1,   72,  496,  175,   14,    1,   26,   30,   36,
            14,   97,   52,  113, 1575, 3576,   19,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [  81,   14,    1,   26,   14, 1640, 1952,   26,  176,   36,   14,    1,
            26, 3105,  196,   37, 1929,   14,  611, 1952,   88,    8, 3106,

In [129]:
model = CBOW(vocab_size=V, emb_size=50)

In [130]:
train_loader = DataLoader(sub_dataset_train, batch_size=500, shuffle=True)

In [131]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        total_loss = 0
        total = 0
        model.train()
        for x, y, s in train_loader:
            x = x.type(torch.LongTensor)  #.cuda()
            y = y.type(torch.FloatTensor).unsqueeze(1)
            s = s.type(torch.Tensor).view(s.shape[0], 1)
            y_hat = model(x, s)
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += x.size(0)*loss.item()
            total += x.size(0)
        train_loss = total_loss/total
        val_loss, val_accuracy = val_metrics(model)
        
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (train_loss, val_loss, val_accuracy))

In [75]:
train_epocs(model, epochs=10)

train_loss 0.672 val_loss 0.626 val_accuracy 0.770
train_loss 0.548 val_loss 0.461 val_accuracy 0.859
train_loss 0.368 val_loss 0.318 val_accuracy 0.890
train_loss 0.250 val_loss 0.259 val_accuracy 0.905
train_loss 0.191 val_loss 0.237 val_accuracy 0.906
train_loss 0.157 val_loss 0.232 val_accuracy 0.907
train_loss 0.132 val_loss 0.230 val_accuracy 0.910
train_loss 0.113 val_loss 0.234 val_accuracy 0.908
train_loss 0.098 val_loss 0.237 val_accuracy 0.907
train_loss 0.085 val_loss 0.244 val_accuracy 0.902
